In [1]:
import os

import numpy as np
import pandas as pd
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

from data import *


SEED = 662
DATA_DIR = "processed"

/Users/borisleung/anaconda3/envs/sta841kaggle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import traceback

import lightgbm as lgb
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsTransformer
from sklearn.pipeline import Pipeline

from models import *


def load_objective(
    X: pd.DataFrame,
    y: pd.DataFrame,
    y_binarized: pd.DataFrame,
    get_trained_model: callable,
):
    X.replace(-999, np.nan, inplace=True)

    def objective(trial: optuna.Trial):
        chi2_threshold = trial.suggest_int("chi2_threshold", 2, 100)
        # cv = trial.suggest_int("cv", 3, 5)
        cv = 3
        imputer_strategy = trial.suggest_categorical(
            "imputer_strategy", ["mean", "median", "most_frequent"]
        )

        train_losses, valid_losses, valid_loss_shift, n_iters = [], [], [], []
        X_train, X_valid, y_train, y_valid, y_train_binarized, y_valid_binarized = (
            train_test_split(X, y, y_binarized, test_size=1 / cv, random_state=SEED)
        )  # lazy, just do 1 round of cv

        # compute pov means aggregated by psu
        aggregated_pov_train_data = pd.concat(
            [X_train["psu_hh_idcode"], y_train], axis=1
        )
        aggregated_pov_train_data[["psu", "hh", "idcode"]] = aggregated_pov_train_data[
            "psu_hh_idcode"
        ].str.split("_", expand=True)
        df_mean = aggregated_pov_train_data.groupby("psu")["num_pov"].mean()
        X_train_means = pd.merge(
            aggregated_pov_train_data,
            df_mean,
            on="psu",
            how="left",
            suffixes=("", "_mean"),
        )["num_pov_mean"]
        X_train_means.replace(np.nan, X_train_means.mean(), inplace=True)

        X_valid[["psu", "hh", "idcode"]] = X_valid["psu_hh_idcode"].str.split(
            "_", expand=True
        )
        X_valid_means = pd.merge(
            X_valid,
            df_mean,
            on="psu",
            how="left",
            suffixes=("", "_mean"),
        )["num_pov"]
        X_valid_means.replace(np.nan, X_valid_means.mean(), inplace=True)
        X_valid_means.rename("num_pov_mean", inplace=True)

        preprocessor = get_preprocessor(
            # ordinal_transformer=Pipeline([("imputer", "passthrough")]),
            imputer_strategy=[
                "most_frequent",
                "most_frequent",
                imputer_strategy,
                imputer_strategy,
            ],
            remainder="drop",
        )

        X_train_processed = preprocessor.fit_transform(X_train)
        selector = SelectPercentile(chi2, percentile=chi2_threshold)
        X_train = selector.fit_transform(X_train_processed, y_train)

        X_valid = preprocessor.transform(X_valid)
        X_valid = selector.transform(X_valid)

        # add pov means to the data
        X_train = np.column_stack([X_train, X_train_means])
        X_valid = np.column_stack([X_valid, X_valid_means])

        model = get_trained_model(
            trial, seed=SEED, X_train=X_train, y_train=y_train - 1
        )
        # train_losses.append(
        #     log_loss(y_train_binarized, model.predict_proba(X_train), normalize=False)
        #     / len(y_train_binarized)
        # )

        y_pred = model.predict_proba(X_valid)
        for column_index in [
            column for column in range(10) if column not in model.classes_
        ]:
            y_pred = np.insert(y_pred, column_index - 1, 0, axis=1)

        valid_losses.append(
            log_loss(y_valid_binarized, y_pred, normalize=False)
            / len(y_valid_binarized)
        )
        # valid_loss_shift.append(valid_losses[-1] - train_losses[-1])
        # n_iters.append(model.n_iter_[0])

        mean_valid_loss = np.mean(valid_losses)
        trial.set_user_attr(
            "selected_columns", X_train_processed.columns[selector.get_support()]
        )
        # trial.set_user_attr("n_iter", np.mean(n_iters))
        # trial.set_user_attr("train_loss", np.mean(train_losses))
        # trial.set_user_attr("valid_loss_shift", np.mean(valid_loss_shift))
        return mean_valid_loss

    return objective


def get_lr(trial, seed, X_train, y_train):
    model = suggest_logistic_regression(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_linear_svc(trial, seed, X_train, y_train):
    model, calibrated_model = suggest_linear_svc(trial, seed)
    model.fit(X_train, y_train)
    calibrated_model.fit(X_train, y_train)
    return calibrated_model


def get_kernel_svc(trial, seed, X_train, y_train):
    model = suggest_kernel_svc(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_naive_bayes(trial, seed, X_train, y_train):
    model = suggest_naive_bayes(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_knn(trial, seed, X_train, y_train):
    model = suggest_knn_classifier(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_random_forest(trial, seed, X_train, y_train):
    model = suggest_random_forest(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_gradient_boosting(trial, seed, X_train, y_train):
    model = suggest_gradient_boosting(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_mlp(trial, seed, X_train, y_train):
    model = suggest_mlp_classifier(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_lightgbm(trial, seed, X_train, y_train):
    callbacks = [lgb.log_evaluation(period=0)]
    model = suggest_lightgbm(trial, SEED)
    model.fit(X_train, y_train, callbacks=callbacks)
    return model


models = {
    "naive_bayes": (get_naive_bayes, 500),
    "knn": (get_knn, 500),
    "lr": (get_lr, 1000),
    "linear_svc": (get_linear_svc, 1000),
    "kernel_svc": (get_kernel_svc, 300),
    "random_forest": (get_random_forest, 800),
    # "gradient_boosting": (get_gradient_boosting, 200),
    # "lightgbm": (get_lightgbm, 200),
    # "mlp": (get_mlp, 200),
}


optuna.logging.set_verbosity(optuna.logging.ERROR)
# data = get_divided_edu(
#     remove_boring_columns(transform_all_house(combined_transformed_train_with_num_pov))
# )
# poi = data[1]
data = remove_boring_columns(
    transform_education_levels(
        transform_all_house(combined_transformed_train_with_num_pov)
    )
)

poi = data
nunique = poi.nunique()
cols_to_drop = nunique[nunique == 1].index
poi = poi.drop(cols_to_drop, axis=1)
poi = poi.drop(columns=["edu_q09", "edu_q10", "edu_q11", "edu_q12", "edu_q13"])
print(poi.shape)
for index, (model_name, (get_trained_model, rounds)) in enumerate(models.items()):
    best_values, best_params = [], []
    print(model_name)
    # if True:
    try:
        X, y, y_binarized = (
            poi.drop(Y_COLUMNS + ["num_pov"], axis=1),
            poi["num_pov"],
            poi[Y_COLUMNS],
        )
        study = optuna.create_study(direction="minimize")
        study.optimize(
            load_objective(X, y, y_binarized, get_trained_model),
            n_trials=rounds,
            n_jobs=-1,
            show_progress_bar=True,
        )
        best_values.append(study.best_value)
        best_params.append(study.best_params)
        print(optuna.importance.get_param_importances(study))
        print(study.best_trial.user_attrs)
    except Exception as e:
        print(traceback.format_exc())
    print(best_values), print(best_params)
    print("=====================================")
    print()
    # break

(5337, 83)
naive_bayes


Best trial: 24. Best value: 1.93201: 100%|██████████| 500/500 [00:35<00:00, 14.27it/s]


{'chi2_threshold': 0.9997000869664292, 'imputer_strategy': 0.00029991303357083997}
{'selected_columns': Index(['ordinal__edu_q04', 'ordinal__edu_q06'], dtype='object')}
[1.9320051055024683]
[{'chi2_threshold': 2, 'imputer_strategy': 'mean'}]

knn


Best trial: 264. Best value: 7.2577: 100%|██████████| 500/500 [00:51<00:00,  9.70it/s] 


{'n_neighbors': 0.9893482778288525, 'chi2_threshold': 0.0053570755765285595, 'imputer_strategy': 0.003007827144815225, 'leaf_size': 0.0011607694646578455, 'weights': 0.000787456344147623, 'algorithm': 0.00028548933702249085, 'p': 5.3104303975665696e-05}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0', 'binary__edu_q03_2.0',
       'binary__edu_q43_2.0', 'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',
       'categorical__edu_q28_infrequent_sklearn',
       'categorical__edu_q42_infrequent_sklearn',
       'categorical__edu_q63_infrequent_sklearn', 'numerical__house_q05y',
       'numerical__house_q09', 'numerical__house_q15', 'numerical__house_q16',
    

...

{'calibration_method': 0.6660714407258032, 'penalty': 0.2224036321739302, 'chi2_threshold': 0.06154957355016025, 'imputer_strategy': 0.03758431285283174, 'intercept_scaling': 0.009449849373525974, 'C': 0.0029411913237487263}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q43_2.0', 'binary__edu_q45_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',
       'categorical__edu_q28_infrequent_sklearn',
       'categorical__edu_q42_infrequent_sklearn',
       'categorical__edu_q63_infrequent_sklearn', 'numerical__house_q05y',
    

Best trial: 219. Best value: 1.73507: 100%|██████████| 300/300 [00:58<00:00,  5.12it/s]


{'C': 0.40089451635310897, 'chi2_threshold': 0.34203790338589446, 'imputer_strategy': 0.1795774833455695, 'kernel': 0.07749009691542709}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q19_2.0', 'binary__edu_q25_2.0',
       'binary__edu_q32_2.0', 'binary__edu_q43_2.0', 'binary__edu_q45_2.0',
       'binary__edu_q46_2.0', 'binary__edu_q50_2.0', 'binary__edu_q57_2.0',
       'binary__edu_q61_2.0', 'binary__edu_q64_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',
       'categorical__edu_q28_infrequent_sklearn',
       'cate

Best trial: 675. Best value: 1.73389: 100%|██████████| 800/800 [06:28<00:00,  2.06it/s]


{'max_features': 0.8369432785239171, 'criterion': 0.07015214857665544, 'max_depth': 0.036150342232137464, 'imputer_strategy': 0.01898745600677429, 'n_estimators': 0.014922982498313838, 'chi2_threshold': 0.012376129043118167, 'min_samples_split': 0.00975360579185538, 'min_samples_leaf': 0.0007140573272284027}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q43_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',
       'categorical__edu_q28_infrequent_sklearn',
       'categorical__edu_q42_infrequent_sklearn',
       'categorical__edu_q63_infrequent_sklearn', 'numerical__house_q0

## with knn means too


In [ ]:
import traceback

import lightgbm as lgb
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsTransformer
from sklearn.pipeline import Pipeline

from models import *


def load_objective(
    X: pd.DataFrame,
    y: pd.DataFrame,
    y_binarized: pd.DataFrame,
    get_trained_model: callable,
):
    X.replace(-999, np.nan, inplace=True)

    def objective(trial: optuna.Trial):
        chi2_threshold = trial.suggest_int("chi2_threshold", 2, 100)
        # cv = trial.suggest_int("cv", 3, 5)
        cv = 3
        imputer_strategy = trial.suggest_categorical(
            "imputer_strategy", ["mean", "median", "most_frequent"]
        )

        train_losses, valid_losses, valid_loss_shift, n_iters = [], [], [], []
        X_train, X_valid, y_train, y_valid, y_train_binarized, y_valid_binarized = (
            train_test_split(X, y, y_binarized, test_size=1 / cv, random_state=SEED)
        )  # lazy, just do 1 round of cv

        # add pov means aggregated by psu
        aggregated_pov_train_data = pd.concat(
            [X_train["psu_hh_idcode"], y_train], axis=1
        )
        aggregated_pov_train_data[["psu", "hh", "idcode"]] = aggregated_pov_train_data[
            "psu_hh_idcode"
        ].str.split("_", expand=True)
        df_mean = aggregated_pov_train_data.groupby("psu")["num_pov"].mean()
        X_train_means = pd.merge(
            aggregated_pov_train_data,
            df_mean,
            on="psu",
            how="left",
            suffixes=("", "_mean"),
        )["num_pov_mean"]
        X_train_means.replace(np.nan, X_train_means.mean(), inplace=True)

        X_valid[["psu", "hh", "idcode"]] = X_valid["psu_hh_idcode"].str.split(
            "_", expand=True
        )
        X_valid_means = pd.merge(
            X_valid,
            df_mean,
            on="psu",
            how="left",
            suffixes=("", "_mean"),
        )["num_pov"]
        X_valid_means.replace(np.nan, X_valid_means.mean(), inplace=True)
        X_valid_means.rename("num_pov_mean", inplace=True)

        preprocessor = get_preprocessor(
            # ordinal_transformer=Pipeline([("imputer", "passthrough")]),
            imputer_strategy=[
                "most_frequent",
                "most_frequent",
                imputer_strategy,
                imputer_strategy,
            ],
            remainder="drop",
        )

        X_train_processed = preprocessor.fit_transform(X_train)
        selector = SelectPercentile(chi2, percentile=chi2_threshold)
        X_train = selector.fit_transform(X_train_processed, y_train)
        X_train = np.column_stack([X_train, X_train_means])

        X_valid = preprocessor.transform(X_valid)
        X_valid = selector.transform(X_valid)
        X_valid = np.column_stack([X_valid, X_valid_means])

        # add knn means too
        p = trial.suggest_int("p", 1, 3)
        k_neighbors = trial.suggest_int("k_neighbors", 2, 15)
        transformer = KNeighborsTransformer(
            mode="connectivity", n_neighbors=k_neighbors, p=p
        )
        X_dist_graph = transformer.fit_transform(X_train)
        X_dist_graph.setdiag(0)

        knn_train_means = np.divide(
            (X_dist_graph @ y_train.T), np.asarray(X_dist_graph.sum(axis=1)).flatten()
        )
        np.nan_to_num(knn_train_means, nan=np.nanmean(knn_train_means), copy=False)
        # print(knn_train_means.shape)
        X_train = np.hstack([X_train, knn_train_means.reshape(-1, 1)])
        knn_valid_means = (
            (transformer.kneighbors_graph(X_valid) @ y_train.T) / k_neighbors
        ).reshape(-1, 1)
        np.nan_to_num(knn_valid_means, nan=np.nanmean(knn_train_means), copy=False)
        X_valid = np.hstack([X_valid, knn_valid_means])

        model = get_trained_model(
            trial, seed=SEED, X_train=X_train, y_train=y_train - 1
        )

        y_pred = model.predict_proba(X_valid)
        for column_index in [
            column for column in range(10) if column not in model.classes_
        ]:
            y_pred = np.insert(y_pred, column_index - 1, 0, axis=1)

        valid_losses.append(
            log_loss(y_valid_binarized, y_pred, normalize=False)
            / len(y_valid_binarized)
        )

        mean_valid_loss = np.mean(valid_losses)
        trial.set_user_attr(
            "selected_columns", X_train_processed.columns[selector.get_support()]
        )
        return mean_valid_loss

    return objective


def get_lr(trial, seed, X_train, y_train):
    model = suggest_logistic_regression(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_linear_svc(trial, seed, X_train, y_train):
    model, calibrated_model = suggest_linear_svc(trial, seed)
    model.fit(X_train, y_train)
    calibrated_model.fit(X_train, y_train)
    return calibrated_model


def get_kernel_svc(trial, seed, X_train, y_train):
    model = suggest_kernel_svc(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_naive_bayes(trial, seed, X_train, y_train):
    model = suggest_naive_bayes(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_knn(trial, seed, X_train, y_train):
    model = suggest_knn_classifier(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_random_forest(trial, seed, X_train, y_train):
    model = suggest_random_forest(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_gradient_boosting(trial, seed, X_train, y_train):
    model = suggest_gradient_boosting(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_mlp(trial, seed, X_train, y_train):
    model = suggest_mlp_classifier(trial, seed)
    model.fit(X_train, y_train)
    return model


def get_lightgbm(trial, seed, X_train, y_train):
    callbacks = [lgb.log_evaluation(period=0)]
    model = suggest_lightgbm(trial, SEED)
    model.fit(X_train, y_train, callbacks=callbacks)
    return model


models = {
    "naive_bayes": (get_naive_bayes, 500),
    "knn": (get_knn, 500),
    "lr": (get_lr, 1000),
    "linear_svc": (get_linear_svc, 1000),
    "kernel_svc": (get_kernel_svc, 300),
    "random_forest": (get_random_forest, 800),
    # "gradient_boosting": (get_gradient_boosting, 200),
    # "lightgbm": (get_lightgbm, 200),
    # "mlp": (get_mlp, 200),
}


optuna.logging.set_verbosity(optuna.logging.ERROR)
# data = get_divided_edu(
#     remove_boring_columns(transform_all_house(combined_transformed_train_with_num_pov))
# )
# poi = data[1]
data = remove_boring_columns(
    transform_education_levels(
        transform_all_house(combined_transformed_train_with_num_pov)
    )
)

poi = data
nunique = poi.nunique()
cols_to_drop = nunique[nunique == 1].index
poi = poi.drop(cols_to_drop, axis=1)
poi = poi.drop(columns=["edu_q09", "edu_q10", "edu_q11", "edu_q12", "edu_q13"])
print(poi.shape)
for index, (model_name, (get_trained_model, rounds)) in enumerate(models.items()):
    best_values, best_params = [], []
    print(model_name)
    # if True:
    try:
        X, y, y_binarized = (
            poi.drop(Y_COLUMNS + ["num_pov"], axis=1),
            poi["num_pov"],
            poi[Y_COLUMNS],
        )
        study = optuna.create_study(direction="minimize")
        study.optimize(
            load_objective(X, y, y_binarized, get_trained_model),
            n_trials=rounds,
            n_jobs=-1,
            show_progress_bar=True,
        )
        best_values.append(study.best_value)
        best_params.append(study.best_params)
        print(optuna.importance.get_param_importances(study))
        print(study.best_trial.user_attrs)
    except Exception as e:
        print(traceback.format_exc())
    print(best_values), print(best_params)
    print("=====================================")
    print()
    # break

(5337, 83)
naive_bayes


Best trial: 331. Best value: 2.09946: 100%|██████████| 500/500 [00:55<00:00,  9.09it/s]


{'chi2_threshold': 0.9996550705495922, 'imputer_strategy': 0.0002167812996432647, 'k_neighbors': 0.00011832405066287784, 'p': 9.82410010157213e-06}
{'selected_columns': Index(['ordinal__edu_q04', 'ordinal__edu_q06'], dtype='object')}
[2.0994553384821413]
[{'chi2_threshold': 2, 'imputer_strategy': 'mean', 'p': 3, 'k_neighbors': 2}]

knn


Best trial: 493. Best value: 7.64746: 100%|██████████| 500/500 [02:33<00:00,  3.26it/s]


{'n_neighbors': 0.9936884402189418, 'chi2_threshold': 0.004151771986442509, 'imputer_strategy': 0.0006739167112011512, 'algorithm': 0.0005989740602310231, 'leaf_size': 0.00032792441940747956, 'weights': 0.0002214156052317465, 'p': 0.00021124690668499572, 'k_neighbors': 0.00012631009185927404}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q25_2.0', 'binary__edu_q32_2.0',
       'binary__edu_q43_2.0', 'binary__edu_q45_2.0', 'binary__edu_q46_2.0',
       'binary__edu_q61_2.0', 'binary__edu_q64_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_in

...

{'calibration_method': 0.6245821590076456, 'chi2_threshold': 0.26241524493330054, 'penalty': 0.0419256217513317, 'p': 0.029535591404372223, 'k_neighbors': 0.024270406806242466, 'imputer_strategy': 0.01540926433345599, 'C': 0.0018330967801917235, 'intercept_scaling': 2.8614983459852876e-05}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q43_2.0', 'binary__edu_q45_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',
       'categorical__edu_q28_infrequent_sklearn',
       'categorical__edu_q42_infrequent_sklearn',
       'categ

Best trial: 100. Best value: 1.73438: 100%|██████████| 300/300 [01:43<00:00,  2.90it/s]


{'kernel': 0.41874915866116397, 'chi2_threshold': 0.2503677307054317, 'imputer_strategy': 0.16770118687261218, 'C': 0.10250927403590651, 'k_neighbors': 0.04888204911252328, 'p': 0.011790600612362203}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q08_2.0',
       'binary__edu_q14_2.0', 'binary__edu_q19_2.0', 'binary__edu_q25_2.0',
       'binary__edu_q32_2.0', 'binary__edu_q43_2.0', 'binary__edu_q45_2.0',
       'binary__edu_q46_2.0', 'binary__edu_q50_2.0', 'binary__edu_q57_2.0',
       'binary__edu_q61_2.0', 'binary__edu_q64_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',

Best trial: 643. Best value: 1.7246: 100%|██████████| 800/800 [07:44<00:00,  1.72it/s] 


{'max_features': 0.6415748343732235, 'chi2_threshold': 0.18659829531072367, 'criterion': 0.07701961160258354, 'max_depth': 0.033714046680846196, 'k_neighbors': 0.022246411179933867, 'n_estimators': 0.01716283707610978, 'min_samples_leaf': 0.012595652715820813, 'imputer_strategy': 0.005084723740954881, 'min_samples_split': 0.0033606133355197384, 'p': 0.0006429739842840673}
{'selected_columns': Index(['binary__house_q02_2.0', 'binary__house_q07_1.0',
       'binary__house_q07_2.0', 'binary__house_q11_2.0',
       'binary__house_q17_1.0', 'binary__edu_q03_2.0', 'binary__edu_q43_2.0',
       'categorical__house_q03_2.0',
       'categorical__house_q03_infrequent_sklearn',
       'categorical__house_q06_4.0',
       'categorical__house_q06_infrequent_sklearn',
       'categorical__edu_q17_13.0', 'categorical__edu_q17_infrequent_sklearn',
       'categorical__edu_q23_infrequent_sklearn',
       'categorical__edu_q28_infrequent_sklearn',
       'categorical__edu_q42_infrequent_sklearn',
     